In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split

In [68]:
df_train = pd.read_csv('train.csv').copy()

In [84]:
# 准备数据
df_train.shape
def train_val_split(train='train.csv',train_file='train_set.csv',val_file='val_set.csv'):
    train_data = pd.read_csv(train)
    train_set, val_set = train_test_split(train_data, random_state=666, test_size=0.2)
    train_set.to_csv(train_file, index=False)
    val_set.to_csv(val_file, index=False)
    print('train_data.shape:',train_data.shape)
    print('train_set.shape:',train_set.shape)
    print('val_set.shape:',val_set.shape)

train_val_split(train='train.csv')

train_data.shape: (42000, 785)
train_set.shape: (33600, 785)
val_set.shape: (8400, 785)


In [93]:
# 标准化
def transform(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5
    x = torch.from_numpy(x)
    return x
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))   #三个颜色通道的平均值，三个颜色通道的标准差
#      #transforms.Flip()  #旋转
#     ]
# )

class MyMNIST(torch.utils.data.Dataset):
    def __init__(self,datatxt,train=True,transform=transform,target_trainform=None):
        self.data = pd.read_csv(datatxt)
        self.transform = transform
        self.train = train
        if self.train:
            self.X = self.data.iloc[:,1:]
            self.X = np.array(self.X)
            self.y = self.data.iloc[:,0]
            self.y = np.array(self.y)
        else:
            self.X=self.data
            self.X=np.array(self.X)
            
    def __getitem__(self,index):
        im = torch.tensor(self.X[index],dtype=torch.float)
        if self.transform is not None:
            im=self.transform(im)
            im=im.reshape([1,28,28])
            if self.train:
                label=torch.tensor(self.y[index],dtype=torch.long)
                return im,label
            else:
                return im
            
    def __len__(self):
        return len(self.data)
    
X_train = MyMNIST(datatxt='train_set.csv',train=True,transform=transform)
X_val = MyMNIST(datatxt='val_set.csv',train=True,transform=transform)
X_test  = MyMNIST(datatxt='test.csv',train=False,transform=transform)

train_data = torch.utils.data.DataLoader(X_train,batch_size=64,shuffle=True)
val_data = torch.utils.data.DataLoader(X_val, batch_size=64, shuffle=False)
test_data = torch.utils.data.DataLoader(X_test, batch_size=1000, shuffle=False)

In [83]:
X_train 

In [86]:
dataiter = iter(val_data) #随机加载一个mini batch
dataiter.next()

[tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]]],
 
 
         [[[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]]],
 
 
         [[[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]]],
 
 
         ...,
 
 
         [[[-1., -1., -1.,  ..., -

In [107]:
class LeNet_simple(nn.Module):
    def __init__(self):
        super(LeNet_simple,self).__init__()
        self.layer1 = nn.Sequential(
            # (N=64,C=1,H=28,W=28)
            nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,padding=1),
            # (N=64,C=6,H=28,W=28)
            nn.MaxPool2d(2,2)
            # (N=64,C=6,H=14,W=14)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,padding=0),
            # (N=64,C=16,H=10,W=10)
            nn.MaxPool2d(2,2)
            # (N=64,C=16,H=5,W=5)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(400,120),
            nn.Linear(120,84),
            nn.Linear(84,10)
        )
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0),-1)
        x = self.layer3(x)
        return x
    
    
def Accuracy(out,labels):
    correct=0.0
    total=0.0
    _,predicted = torch.max(out.data, 1)
    correct += (predicted==labels).sum()
    total += labels.size(0)
    accuracy = correct/total
    return accuracy

    
########k折划分############        
# def get_k_fold_data(k, i, X, y):
#     assert k > 1
#     fold_size = X.shape[0] // k  # 每份的个数
    
#     X_train, y_train = None, None
#     for j in range(k):
#         idx = slice(j * fold_size, (j + 1) * fold_size)  #slice(start,end,step)切片函数
#         ##idx 为每组 valid
#         X_part, y_part = X[idx, :], y[idx]
#         if j == i: ###第i折作valid
#             X_valid, y_valid = X_part, y_part
#         elif X_train is None:
#             X_train, y_train = X_part, y_part
#         else:
#             X_train = torch.cat((X_train, X_part), dim=0) #dim=0增加行数，竖着连接
#             y_train = torch.cat((y_train, y_part), dim=0)
#     return X_train, y_train, X_valid,y_valid


########k折划分-mini_batch############        
# def get_k_fold_data(k, i, X, y):
#     assert k > 1
#     X=X.numpy()
#     fold_size = X.shape[0] // k  # 每份的个数
    
#     X_train = None
#     for j in range(k):
#         idx = slice(j * fold_size, (j + 1) * fold_size)  #slice(start,end,step)切片函数
#         ##idx 为每组 valid
#         X_part = X[idx, :]
#         if j == i: ###第i折作valid
#             X_valid = X_part
#         elif X_train is None:
#             X_train = X_part
#         else:
#             X_train = torch.cat((X_train, X_part), dim=0) #dim=0增加行数，竖着连接
#     return torch.tensor(X_train), torch.tensor(X_valid)
    
net = LeNet_simple()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.001,betas=(0.9,0.99)) #lr学习速率

i=0
for epoch in range(3):
#     train, valid = get_k_fold_data(10,epoch,train_data,None)
    train_loss=[]
    accuracy=0.
    correct=0.
    total=0.
    
    for mini_batch,labels in train_data:
        """训练"""
        optimizer.zero_grad()
        out = net(mini_batch)
        loss = criterion(out,labels)
        loss.backward()
        optimizer.step()
        
        # 输出信息
        i+=1
        accuracy=Accuracy(out,labels)
        if i%100==0:
            print("第{}次：Loss：{} ; Accuracy：{}".format(i,loss,accuracy))
            train_loss.append((i,loss))
    
    for mini_batch,labels in val_data:
        """验证"""
        with torch.no_grad():
            val_out = net(mini_batch)
            val_loss = criterion(val_out,labels)
            accuracy=Accuracy(val_out,labels)
                
    print("准确度为{}.".format(accuracy))

第100次：Loss：0.2752797305583954 ; Accuracy：0.921875
第200次：Loss：0.18505626916885376 ; Accuracy：0.9375
第300次：Loss：0.07753101736307144 ; Accuracy：0.984375
第400次：Loss：0.09379924088716507 ; Accuracy：0.953125
第500次：Loss：0.13330043852329254 ; Accuracy：0.96875
准确度为0.9375.
第600次：Loss：0.08283726871013641 ; Accuracy：0.984375
第700次：Loss：0.19425137341022491 ; Accuracy：0.9375
第800次：Loss：0.1694011092185974 ; Accuracy：0.953125
第900次：Loss：0.0904674381017685 ; Accuracy：0.984375
第1000次：Loss：0.05042215436697006 ; Accuracy：0.984375
准确度为1.0.
第1100次：Loss：0.12709277868270874 ; Accuracy：0.953125
第1200次：Loss：0.08249735832214355 ; Accuracy：0.984375
第1300次：Loss：0.05995398759841919 ; Accuracy：0.96875
第1400次：Loss：0.010672450065612793 ; Accuracy：1.0
第1500次：Loss：0.03863348439335823 ; Accuracy：1.0
准确度为1.0.


In [125]:
id=1
image_id=[]
result=[]

for mini_batch in test_data:
    """测试"""
    test_out = net(mini_batch)
    _,predicted = torch.max(test_out.data, 1)
    predicted=predicted.tolist()
    result.extend(predicted)
    for i in predicted:
        image_id.append(id)
        id+=1

csv=pd.DataFrame({'ImageId':image_id,'Label':result})
print(csv)
csv.to_csv('predcit1.csv',index=None)

       ImageId  Label
0            1      2
1            2      0
2            3      9
3            4      9
4            5      3
...        ...    ...
27995    27996      9
27996    27997      7
27997    27998      3
27998    27999      9
27999    28000      2

[28000 rows x 2 columns]
